In [ ]:
!python -V

# <center> GENERADORES</center>
# <center> YIELD</center>

In [421]:
def iterador_generadores(generador):

    while generador:        
        try:
            print(next(generador))
        except StopIteration:
            print('Finalizo iteracion')
            break

def lenguaje():
    yield 'Python'
    yield 'Java'
    yield 'Scala'
               
print(list(lenguaje()))

['Python', 'Java', 'Scala']


In [422]:
iterador_generadores(lenguaje())

Python
Java
Scala
Finalizo iteracion


# <center> YIELD FROM</center>

In [423]:
def lista_generador():
    
    yield from 'JorgeCardona'
    
iterador_generadores(lista_generador())

J
o
r
g
e
C
a
r
d
o
n
a
Finalizo iteracion


# <center> **YIELD & YIELD FROM** DENTRO DE OTRO YIELD</center>

In [424]:
def astronomia():
    yield 'Sol'
    yield 'Luna'
    yield 'Estrellas'
    yield 'Galaxias'
    yield 'Planetas'   

def numeros():
    yield 'Uno'
    yield 'Dos'
    yield lenguaje()
    yield from astronomia()
    yield 'Tres'
    yield 'Cuatro'
    yield lista_generador()
    yield 'Cinco'


print(list(numeros()))

['Uno', 'Dos', <generator object lenguaje at 0x7f06819c1d20>, 'Sol', 'Luna', 'Estrellas', 'Galaxias', 'Planetas', 'Tres', 'Cuatro', <generator object lista_generador at 0x7f06819c1bd0>, 'Cinco']


# <center> **.\__\___next\__\___()**</center>

In [425]:
def iterador_tareas(lista_funciones):    
    # itera la lista de funciones con generadores
    while lista_funciones:
        
        print(lista_funciones, len(lista_funciones))
        # obtiene la funcion que tiene generadores
        actual = lista_funciones.pop(0)
        
        try:
            print(actual.__next__())
            lista_funciones.append(actual)
            
        except Exception:
            # para evaluar si es de tipo Generator
            import types
            print(f"{actual} -> {'Generador sin elementos' if isinstance(actual, types.GeneratorType) else 'No es una expresion Generadora'} , es una  {type(actual)}")
            pass
        
        
iterador_tareas([lista_generador(), lenguaje(), astronomia(), numeros()])

[<generator object lista_generador at 0x7f06819c1d20>, <generator object lenguaje at 0x7f06819c1bd0>, <generator object astronomia at 0x7f06819c1230>, <generator object numeros at 0x7f06819c1cb0>] 4
J
[<generator object lenguaje at 0x7f06819c1bd0>, <generator object astronomia at 0x7f06819c1230>, <generator object numeros at 0x7f06819c1cb0>, <generator object lista_generador at 0x7f06819c1d20>] 4
Python
[<generator object astronomia at 0x7f06819c1230>, <generator object numeros at 0x7f06819c1cb0>, <generator object lista_generador at 0x7f06819c1d20>, <generator object lenguaje at 0x7f06819c1bd0>] 4
Sol
[<generator object numeros at 0x7f06819c1cb0>, <generator object lista_generador at 0x7f06819c1d20>, <generator object lenguaje at 0x7f06819c1bd0>, <generator object astronomia at 0x7f06819c1230>] 4
Uno
[<generator object lista_generador at 0x7f06819c1d20>, <generator object lenguaje at 0x7f06819c1bd0>, <generator object astronomia at 0x7f06819c1230>, <generator object numeros at 0x7f068

# <center> **ENVIAR VALORES A UN GENERADOR**</center>

In [410]:
def multiplicar_valor():
    while True:
        x,y = yield
        yield x + y
gen = multiplicar_valor()

In [411]:
for i in range(10,20):
    next(gen)
    resultado = gen.send([i//2,i])
    print(resultado)

15
16
18
19
21
22
24
25
27
28


# <center> **ENVIAR VALORES A UN GENERADOR CON CONDICIONALES**</center>

In [412]:
def multiplicar_valor(generador):
    num = 0
    while generador:
        actual = next(generador)
        
        yield actual
        
        if actual%2 !=0:
            
            # parametros a capturar con el metodo send
            x,y = yield
            
            # define las variables a capturar
            if (x and y) is not None:
                yield f'el valor {x*actual} + {y*actual*2} = {x*actual + y*actual*2}'
        
            
iterador = (i for i in range(10))
gen = multiplicar_valor(iterador)

In [413]:
for i in range(99):
    
    try:
        next(gen)
        resultado_parametros = gen.send([i+2,i+5])
        print(resultado_parametros)
    except Exception:
        print('Iteracion Finalizada')
        break

1
el valor 3 + 12 = 15
3
el valor 15 + 48 = 63
5
el valor 35 + 100 = 135
7
el valor 63 + 168 = 231
9
el valor 99 + 252 = 351
Iteracion Finalizada


# <center> **EXCEPCIONES EN UN GENERADOR**</center>

In [414]:
numeros = (valor for valor in range(10))

def iterador_generadores(generador):

    while generador:        
        try:
            
            valor = next(generador)
            print(valor)
            
            # criterio de parada
            if valor > 5:
                generador.throw(ValueError("EL VALOR HA SUPERADO EL LIMITE PERMITIDO"))
                
        except StopIteration:
            print('Finalizo iteracion')
            break
            
iterador_generadores(numeros)

0
1
2
3
4
5
6


ValueError: EL VALOR HA SUPERADO EL LIMITE PERMITIDO

# <center> **DETENER O FINALIZAR UN GENERADOR**</center>

In [415]:
numeros = (valor for valor in range(10))

def iterador_generadores(generador):

    while generador:        
        try:
            
            valor = next(generador)
            print(valor)
            
            # criterio de parada
            if valor > 5:
                generador.close()
                
        except StopIteration:
            print('Finalizo iteracion')
            break

iterador_generadores(numeros)

0
1
2
3
4
5
6
Finalizo iteracion


# <center> **PIPE O CANALIZACIONES CON GENERADORES**</center>

In [429]:
def fibonacci(nums):
    x, y = 0, 1
    for _ in range(nums):
        x, y = y, x+y
        yield x

def cuadrado(nums):
    for num in nums:
        yield num**2

print(list(cuadrado(fibonacci(10))))
print(sum(cuadrado(fibonacci(10))))

[1, 1, 4, 9, 25, 64, 169, 441, 1156, 3025]
4895


In [ ]:
import asyncio
async def num_calc(name, number):
    f = 1
    for i in range(2, number + 1):
        await asyncio.sleep(1)
        f *= i
    print(f"Task {name}: multiplier({number}) = {f}")
    return sum(i * i for i in range(10 **(f + 5)))
async def main():
    # schedule calls concurrently & gathers all async future results
    results = await asyncio.gather(num_calc("A", 2), num_calc("B", 3), num_calc("C", 4) )  
    print(results)    

await main()

Task A: multiplier(2) = 2
Task B: multiplier(3) = 6


# <center> **CONSUMO DE MEMORIA EN BYTES DE UNA LISTA VS UN GENERADOR**</center>

In [79]:
import sys
lista = [i * 2 for i in range(10000)]
print(sys.getsizeof(lista))

85176


In [80]:
generador = (i ** 2 for i in range(10000))

while generador: 

    try:
        siguiente = next(generador)
        print(sys.getsizeof(siguiente), sys.getsizeof(generador), siguiente)
    except StopIteration:
        print('Finalizo iteracion')
        break

24 104 0
28 104 1
28 104 4
28 104 9
28 104 16
28 104 25
28 104 36
28 104 49
28 104 64
28 104 81
28 104 100
28 104 121
28 104 144
28 104 169
28 104 196
28 104 225
28 104 256
28 104 289
28 104 324
28 104 361
28 104 400
28 104 441
28 104 484
28 104 529
28 104 576
28 104 625
28 104 676
28 104 729
28 104 784
28 104 841
28 104 900
28 104 961
28 104 1024
28 104 1089
28 104 1156
28 104 1225
28 104 1296
28 104 1369
28 104 1444
28 104 1521
28 104 1600
28 104 1681
28 104 1764
28 104 1849
28 104 1936
28 104 2025
28 104 2116
28 104 2209
28 104 2304
28 104 2401
28 104 2500
28 104 2601
28 104 2704
28 104 2809
28 104 2916
28 104 3025
28 104 3136
28 104 3249
28 104 3364
28 104 3481
28 104 3600
28 104 3721
28 104 3844
28 104 3969
28 104 4096
28 104 4225
28 104 4356
28 104 4489
28 104 4624
28 104 4761
28 104 4900
28 104 5041
28 104 5184
28 104 5329
28 104 5476
28 104 5625
28 104 5776
28 104 5929
28 104 6084
28 104 6241
28 104 6400
28 104 6561
28 104 6724
28 104 6889
28 104 7056
28 104 7225
28 104 7396
28

# <center> **VELOCIDAD DE PROCESAMIENTO VS CONSUMO DE MEMORIA DE UNA LISTA VS UN GENERADOR**</center>

In [86]:
import cProfile

cProfile.run('sum([i * 2 for i in range(10000)])')

         5 function calls in 0.025 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.023    0.023    0.023    0.023 <string>:1(<listcomp>)
        1    0.001    0.001    0.024    0.024 <string>:1(<module>)
        1    0.000    0.000    0.025    0.025 {built-in method builtins.exec}
        1    0.001    0.001    0.001    0.001 {built-in method builtins.sum}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [87]:
cProfile.run('sum((i * 2 for i in range(10000)))')

         10005 function calls in 0.081 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10001    0.045    0.000    0.045    0.000 <string>:1(<genexpr>)
        1    0.000    0.000    0.081    0.081 <string>:1(<module>)
        1    0.000    0.000    0.081    0.081 {built-in method builtins.exec}
        1    0.036    0.036    0.081    0.081 {built-in method builtins.sum}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




# <center> iter()</center>

In [416]:
iterador = iter([1,2,3,4,5,6,7,8,9,0])

In [417]:
print(dir(iterador))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__length_hint__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__']


In [418]:
iterador.__next__()

1

In [419]:
def is_palindrome(num):
    
    r = str(abs(num)) if type(int) else num

    if len(r) < 2:
        return False
    
    elif len(r) %2 == 0:
        size = len(r)//2
        a = r[:size]
        b = r[size:][::-1]
        
        return a==b
        

    else:
        size = len(r)//2
        a = r[:size]
        b = r[size+1:][::-1]
        
        return a==b
        
listado = list()

for i in range(-1000,1000):

    if (is_palindrome(i)):
        listado.append(i)

print(listado)

[-999, -989, -979, -969, -959, -949, -939, -929, -919, -909, -898, -888, -878, -868, -858, -848, -838, -828, -818, -808, -797, -787, -777, -767, -757, -747, -737, -727, -717, -707, -696, -686, -676, -666, -656, -646, -636, -626, -616, -606, -595, -585, -575, -565, -555, -545, -535, -525, -515, -505, -494, -484, -474, -464, -454, -444, -434, -424, -414, -404, -393, -383, -373, -363, -353, -343, -333, -323, -313, -303, -292, -282, -272, -262, -252, -242, -232, -222, -212, -202, -191, -181, -171, -161, -151, -141, -131, -121, -111, -101, -99, -88, -77, -66, -55, -44, -33, -22, -11, 11, 22, 33, 44, 55, 66, 77, 88, 99, 101, 111, 121, 131, 141, 151, 161, 171, 181, 191, 202, 212, 222, 232, 242, 252, 262, 272, 282, 292, 303, 313, 323, 333, 343, 353, 363, 373, 383, 393, 404, 414, 424, 434, 444, 454, 464, 474, 484, 494, 505, 515, 525, 535, 545, 555, 565, 575, 585, 595, 606, 616, 626, 636, 646, 656, 666, 676, 686, 696, 707, 717, 727, 737, 747, 757, 767, 777, 787, 797, 808, 818, 828, 838, 848, 858

# <center> TIPOS DE INSTANCIA</center>

# <center> SINCRONISMO - HILOS - PROCESOS - ASINCRONISMO</center>

<center>
    <img width="50%" lign="center" src="ejecuciones.png">
</center>

# <center> EJECUCION **SINCRONA**</center>

In [ ]:
%%time

from time import sleep
espera = 3


def funcion(x:int) -> str:    
    sleep(x)
    print(f' procesado {x}')
    return f'termina funcion {x}'

datos = []

for tiempo_espera in range(1,espera + 1):
    datos.append(funcion(tiempo_espera))
    
datos

# <center> EJECUCION **ASINCRONA**</center>

In [ ]:
from datetime import datetime
from time import sleep
import asyncio

async def funcion(x:int) -> str:    
    await asyncio.sleep(x)
    print(f' procesado {x}')
    return f'termina funcion {x}'

datos = []

inicia = datetime.now()


for tiempo_espera in range(1,espera + 1):
    await datos.append(await funcion(tiempo_espera))

finaliza = datetime.now()
print(finaliza.second - inicia.second ) 
datos

# <center> EJECUCION **HILO**</center>
## <center> Este módulo construye interfaces de hilado de alto nivel sobre el módulo de más bajo nivel _thread. 

In [ ]:
%%time

def funcion(x:int) -> str:    
    sleep(x)
    print(f' procesado {x}')
    return f'termina funcion {x}'

# libreria para procesamiento en concurrente
from threading import Thread

hilos = []
for tiempo_espera in range(1,espera + 1):

    t = Thread(name=f'ejecucion de tarea # {tiempo_espera}', target=funcion, args=(tiempo_espera,))
    
    t.start()    
    hilos.append(t)
    
    
for t in hilos:
    t.join()

hilos

# <center> EJECUCION **PROCESO**</center>
## <center> Este módulo permite crear procesos (spawning) utilizando una API similar al módulo threading. ofrece concurrencia tanto local como remota. permite aprovechar al máximo múltiples procesadores en una máquina determinada.

In [ ]:
%%time

def funcion(x:int) -> str:    
    sleep(x)
    print(f' procesado {x}')
    return f'termina funcion {x}'

# importa las librerias que permiten usar la computacion paralela
from multiprocessing import Process


procesos = []
for tiempo_espera in range(1,espera + 1):
    t = Process(name=f'ejecucion de tarea # {tiempo_espera}', target=funcion, args=(tiempo_espera,))
    t.start()    
    procesos.append(t)
    
for t in procesos:
    t.join()

procesos

# <center> EJECUCION 3600 TAREAS DE 2 SEGUNDOS, TOMARIA 2 HORAS EN EJECUCION SINCRONA</center>

In [ ]:
%%time

def funcion(x:int) -> str:    
    sleep(x)
    return f'termina funcion {x}'

ejecuciones = []
for tiempo_espera in range(1,3601):

    t = Thread(name=f'ejecucion de tarea # {tiempo_espera}', target=funcion, args=(2,))
    
    t.start()    
    ejecuciones.append(t)
    
for t in ejecuciones:
    t.join()

ejecuciones

In [ ]:
%%time

ejecuciones = []
for tiempo_espera in range(1,3601):

    t = Process(name=f'ejecucion de tarea # {tiempo_espera}', target=funcion, args=(2,))
    
    t.start()    
    ejecuciones.append(t)
    
for t in ejecuciones:
    t.join()

ejecuciones

# <center> FUNCIONES PARA LOS HILOS Y PROCESOS</center>

In [ ]:
def generar_instancia(tipo_tarea=None, funcion_a_procesar=None, **kw):
    
    # convierte los argumentos a tupla ya que los argumentos deben ser de tipo tupla
    argumentos = tuple(kw.values())
    
    # tipos de Instancia que se van a usar
    tipo_instancia = {
            'hilos':Thread,
            'procesos':Process
            }
        
    # crea la instancia
    tipo_ejecucion = tipo_instancia.get(tipo_tarea, Thread)
    
    # retorna la instancia con los argumentos a evaluar
    return tipo_ejecucion(name='test_ejecucion', target=funcion_a_procesar, args=argumentos)


def iniciar_instancia(instancia):
    
    # muestra el nombre del hilo o del proceso
    print('tarea inicializada ', instancia.name)
    
    # inicializa la ejecucion de la instruccion invocada en el hilo o proceso
    instancia.start()
    
    return instancia

def finalizar_instancia(instancia):

    # garantiza que se finaliza la funcion invocada, antes de que se continue con una nueva instruccion
    instancia.join()
    
    # muestra el nombre del hilo o del proceso
    print('tarea Finalizada', instancia.name)
    
    return instancia

# <center> HILOS</center>
## <center>Es una linea de ejecucion de un proceso, Para ejecuciones a gran escala permite delegar acciones en tareas mas pequeñas</center>

## <center> un Lock es un mecanismo de sincronización para forzar el acceso a una sección de código en un programa. por ejemplo </center>

# <center> COLA PARA ADICIONAR RESULTADOS HILOS</center>

In [ ]:
# libreria para procesamiento en paralelo
from threading import Thread

# libreria que permite guardar los valores retornados en una cola
from queue import Queue as cola_classica

# instancia de la cola para adicionar los resultados
almacenar_resultados = cola_classica()

def almacenar_en_cola(f):
    def wrapper(*args):
        almacenar_resultados.put(f(*args))
    return wrapper

@almacenar_en_cola
def retornar_lista(numero=100):
    return [i for i in range(numero)]

def obtener_resultado_cola_tarea(cola):
    
    for indice_tarea in range(cola.qsize()):
        print(cola.get(indice_tarea))

In [ ]:
listado_instancias = []
   
    
for cantidad_instancias in range(5):
  
    instancia = generar_instancia(tipo_tarea='hilos', funcion_a_procesar=retornar_lista, numero=20)

    ejecutar_tarea = iniciar_instancia(instancia)
   
    listado_instancias.append(ejecutar_tarea)


for tipo_instancia in listado_instancias:
    
    finalizar_instancia(tipo_instancia)

obtener_resultado_cola_tarea(almacenar_resultados)

# <center> PROCESOS</center>
## <center> Process-based parallelism, estan compuestos de multiples hilos, utilizan todos los nucleos del procesador que se puedan usar.</center>

# <center> COLA PARA ADICIONAR RESULTADOS PROCESOS</center>

In [ ]:
# importa las librerias que permiten usar la computacion paralela
from multiprocessing import Process

# libreria que permite guardar los valores retornados en una cola
from multiprocessing import Queue as cola_multiproceso

# instancia de la cola para adicionar los resultados
almacenar_resultados_multiproceso = cola_multiproceso()

def almacenar_en_cola_multiproceso(f):
    def wrapper(*args):
        almacenar_resultados_multiproceso.put(f(*args))
    return wrapper

@almacenar_en_cola_multiproceso
def retornar_lista_proceso(numero=20):
    return [chr(i + 64)  for i in range(numero)]

def obtener_resultado_cola_tarea_while(cola):
    
    while not cola.empty():
        result = cola.get()
        print (result)

In [ ]:
listado_instancias = []

for cantidad_instancias in range(5):
  
    instancia = generar_instancia(tipo_tarea='procesos', funcion_a_procesar=retornar_lista_proceso, numero=10)

    ejecutar_tarea = iniciar_instancia(instancia)
   
    listado_instancias.append(ejecutar_tarea)


for tipo_instancia in listado_instancias:
    
    finalizar_instancia(tipo_instancia)

obtener_resultado_cola_tarea_while(almacenar_resultados_multiproceso)

# <center> POOL DE PROCESOS E HILOS</center>
## <center>Ofrece un medio conveniente de paralelizar la ejecución de una función a través de múltiples valores de entrada, distribuyendo los datos de entrada a través de procesos (paralelismo de datos).</center>

In [ ]:
from multiprocessing.pool import Pool, ThreadPool

def calcular_cuadrado(x):
    return x*x

### <center> APPLY - PROCESOS</center>

In [ ]:
pool = Pool(processes=2)
pool_apply = pool.apply(calcular_cuadrado, [5])
pool.close() # Prevents any more tasks from being submitted to the pool. Once all the tasks have been completed the worker processes will exit.
pool.join() # Wait for the worker processes to exit. One must call close() or terminate() before using join().
pool.close() # Close the Process object, releasing all resources associated with it.
pool_apply

### <center> APPLY - HILOS</center>

In [ ]:
pool = ThreadPool(processes=2)
pool_apply = pool.apply(calcular_cuadrado, [7])
pool.close() # Prevents any more tasks from being submitted to the pool. Once all the tasks have been completed the worker processes will exit.
pool.join() # Wait for the worker processes to exit. One must call close() or terminate() before using join().
pool.close() # Close the Process object, releasing all resources associated with it.
pool_apply

### <center> APPLY_ASYNC</center>

In [ ]:
pool = Pool(processes=2)
pool_apply_async = pool.apply_async(calcular_cuadrado, [3])
pool.close()
pool.join()
pool.close()
pool_apply_async.get()

### <center> MAP_ASYNC</center>

In [ ]:
pool = Pool(processes=2)
pool_map_async = pool.map_async(calcular_cuadrado, range(0,20,3))
pool.close()
pool.join()
pool.close()
pool_map_async.get()

### <center> MAP</center>

In [ ]:
pool = Pool(processes=2)
pool_map = pool.map(calcular_cuadrado, range(0,20,5))
pool.close()
pool.join()
pool.close()
pool_map

### <center> IMAP</center>

In [ ]:
pool = Pool(processes=2)
pool_imap_async = pool.imap(calcular_cuadrado, range(0,20,4))
pool.close()
pool.join()
pool.close()
list(pool_imap_async)

### <center> IMAP_UNORDERED</center>

In [ ]:
pool = Pool(processes=2)
pool_imap_unordered = pool.imap_unordered(calcular_cuadrado, range(0,20,2))
pool.close()
pool.join()
pool.close()
list(pool_imap_unordered)

### <center> STARMAP</center>

In [ ]:
pool = Pool(processes=2)
pool_starmap = pool.starmap(calcular_cuadrado, [[1],[2],[3],[4],[5]])
pool.close()
pool.join()
pool.close()
list(pool_starmap)

### <center> STARMAP_ASYNC</center>

In [ ]:
pool = Pool(processes=2)
pool_starmap_async = pool.starmap_async(calcular_cuadrado, [[1],[2],[3],[4],[5]])
pool.close()
pool.join()
pool.close()
pool_starmap_async.get()

# <center> PROCESAMIENTO MASIVO</center>
## <p> Lanzamiento de tareas paralelas, El módulo concurrent.futures provee una interfaz de alto nivel para ejecutar **invocables de forma asincrónica**. La ejecución asincrónica se puede realizar mediante hilos, usando ThreadPoolExecutor, o procesos independientes, mediante ProcessPoolExecutor. Ambos implementan la misma interfaz, que se encuentra definida por la clase abstracta Executor.</p>

In [ ]:
# importa librerias para trabajar concurrencia
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor

# tipos de Instancia que se van a usar
tipo_instancia = {
        'hilos':Thread,
        'procesos':Process
        }

# adiciona nuevas key/values al diccionario
tipo_instancia['multihilos'] = ThreadPoolExecutor
tipo_instancia['multiproceso'] = ProcessPoolExecutor

tipo_instancia

In [ ]:
import random

def generar_lista(numero = 22):
    
    return [random.randint(-100, i**2) for i in range(numero)]
    
def imprimir_multi_datos(lista):
    
    for valores in lista:
        print(valores)
        
def generar_pool_iniciar_y_finalizar_multi_instancia(tipo_tarea=None, funcion_a_procesar=None, numero = 22):
    
    # crea la instancia de ejecucion
    tipo_ejecucion = tipo_instancia.get(tipo_tarea, ThreadPoolExecutor)
    
    # especifica el numero de hilos o procesos que se quieren usar como base, workers basado en el total de CPUs de la maquina
    pool = tipo_ejecucion(max_workers=nucleos_totales)
    
    # recupera la informacion de la tarea que se ejecuta
    response = pool.submit(funcion_a_procesar, numero)
    
    # finaliza el pool
    pool.shutdown()
    
    # obtiene la respuesta de la tarea
    return response.result()

# <center> MULTI HILOS</center>
# <center> interruptores rápidos b/w que dan la ilusion de multi tarea</center>

In [ ]:
listado = []
for i in range(5):
    instancia = generar_pool_iniciar_y_finalizar_multi_instancia(tipo_tarea='multihilos', funcion_a_procesar=generar_lista, numero= i + 1)
    # almacena las tareas ejecutadas
    listado.append(instancia)

imprimir_multi_datos(listado)

# <center> MULTI PROCESOS</center>

In [ ]:
listado = []
for i in range(5):
    instancia = generar_pool_iniciar_y_finalizar_multi_instancia(tipo_tarea='multiprocesos', funcion_a_procesar=generar_lista, numero= i + 1)
    # almacena las tareas ejecutadas
    listado.append(instancia)

imprimir_multi_datos(listado)

# <center> MULTI HILOS Y MULTIPROCESOS CON MULTIPLES ARGUMENTOS USANDO **MAP**</center>

In [ ]:
def cuadrado(x=9):
    
    return x**2

def multi_argumentos(*parametros, tipo_tarea=None, funcion_a_procesar=None):
    
    # convierte los parametros tipo tupla a lista
    argumentos = list(parametros[0])
    
    # crea la instancia de ejecucion
    tipo_ejecucion = tipo_instancia.get(tipo_tarea, ThreadPoolExecutor)
    
    # asigna los workers basado en el total de CPUs de la maquina
    pool = tipo_ejecucion(max_workers=nucleos_totales)
        
    # obtiene los resultados de cada argumento en la funcion  
    response = pool.map(cuadrado, argumentos)
        
    # obtiene la respuesta de la tarea
    return [valor for valor in response]

print(multi_argumentos([1,2,3,4],tipo_tarea='multihilos', funcion_a_procesar=cuadrado))
print(multi_argumentos([5,6,7,8,9],tipo_tarea='multiprocesos', funcion_a_procesar=cuadrado))

In [ ]:
import threading
import time


def thread_x():
    print('Start ', threading.current_thread().name)
    time.sleep(1)
    print('Finish ', threading.current_thread().name)


def thread_y():
    print('Start ', threading.current_thread().name)
    time.sleep(1)
    print('Finish ', threading.current_thread().name)

x = threading.Thread(target=thread_x, name='Thread-X', daemon = True)
y = threading.Thread(target=thread_y, name='Thread-Y')


x.start()
y.start()

x.join()
y.join()

# <center> ASINCRONISMO - ASYNCIO</center>

# <center> EVENT LOOP</center>

<p align="center" width="100%">
    <img width="100%" src="event-loop.png">
</p>

# <p> asyncio es una biblioteca para escribir código concurrente utilizando la sintaxis async/await.Es utilizado como base en múltiples frameworks asíncronos de Python y provee un alto rendimiento en redes y servidores web, bibliotecas de conexión de base de datos, colas de tareas distribuidas, etc. </p>

- asyncio provee un conjunto de APIs de alto nivel para:

- ejecutar corutinas de Python de manera concurrente y tener control total sobre su ejecución;

- realizar redes E/S y comunicación entre procesos(IPC);

- controlar subprocesos;

- distribuir tareas a través de colas;

- sincronizar código concurrente;


- Adicionalmente, existen APIs de bajo nivel para desarrolladores de bibliotecas y frameworks para:

- crear y administrar bucles de eventos, los cuales proveen APIs asíncronas para redes, ejecutando subprocesos, gestionando señales del sistema operativo, etc;

- implementar protocolos eficientes utilizando transportes;

- Bibliotecas puente basadas en retrollamadas y código con sintaxis async/wait.

In [ ]:
import asyncio

# esta seria la forma de ejecutar en un IDE, dado que  jupyter ya esta corriendo un event loop
# siempre es necesario ejecutar una funcion asincrona de esta manera 
#asyncio.run(funcion_asincrona())

# <center> TODA FUNCION ASINCRONA DEBE SER PRECEDIDA POR LA PALABRA **ASYNC**</center>

In [ ]:
async def listado(limite = 10):
    
    return [random.randrange(-100, 100) for x in range(limite)]

In [ ]:
def ejecuta_funcion_listado(limite = 15):
    return listado(limite)

In [ ]:
async def ejecuta_funcion_asincrona(limite = 20):
    return await listado(limite)

# <center> TODA VARIABLE O FUNCION QUE LLAME UNA FUNCION ASYNCRONA DEBE SER PRECEDIDA POR LA PALABRA **AWAIT**</center>

In [ ]:
ejecuta_funcion_listado()

In [ ]:
respuesta = listado()
respuesta

In [ ]:
await respuesta

In [ ]:
await listado()

In [ ]:
await ejecuta_funcion_listado()

In [ ]:
print(await ejecuta_funcion_asincrona())

In [ ]:
lista = []

for i in range(10):
    
    lista.append(await listado())

lista

# <center> **FUNCIONES ASINCRONAS**</center>

In [ ]:
async def Primero(limite=5):
    print("Inicio funcion Primero")
    await asyncio.sleep(2)
    print("Final funcion Primero")
    
    return [random.randrange(-100, 0) for x in range(limite)]

async def Ultimo(limite=5):
    print("Comenzar funcion ultimo")
    await asyncio.sleep(2)
    print("Terminar funcion ultimo")
    
    return [random.randrange(0, 100) for x in range(limite)]

# <center> **GATHER**</center>
## <center> **RECIBE UNA LISTA DE TAREAS Y RETORNA UNA LISTA DE RESULTADOS**</center>
## <center> **TIENE OPCIONES ESPECÍFICAS PARA EL MANEJO DE ERRORES Y CANCELACIONES.**</center>

In [ ]:
# El siguiente ejemplo muestra cómo esperar a que se completen varias tareas asincrónicas.

from asyncio import gather

async def iniciar_gather():
    
    tareas = [Primero(), Ultimo(), Primero(), Ultimo(), Primero(), Ultimo(), Primero()]
    
    return await gather(*tareas)
    
await iniciar_gather()

# <center> **WAIT_FOR**</center>
## <center> **PERMITE DEFINIR EL MAXIMO TIEMPO DE ESPERA DE UNA TAREA**</center>

In [ ]:
# El siguiente ejemplo demuestra cómo podemos utilizar un tiempo de espera para evitar esperar indefinida a que finalice una tarea asincrónica.

from asyncio import wait_for

async def iniciar_wait_for():
    try:
        return await wait_for(Primero(), timeout=1)
    except asyncio.TimeoutError:
        print("la ejecucion supero el tiempo de espera!")
        

await iniciar_wait_for()

# <center> **AS_COMPLETED**</center>
## <center> **ES SIMILIAR A GATHER, PERO RETORNA FUTUROS, LOS RESULTADOS SON RETORNADOS EN EL ORDEN QUE ESTAN LISTOS**</center>

In [ ]:
# El siguiente ejemplo demuestra cómo as_complete, completará la primera tarea, seguida de la siguiente más rápida y la siguiente hasta que se completen todas las tareas.

from asyncio import as_completed

async def iniciar_as_completed():
    
    tareas = [Primero(), Ultimo(), Primero(), Ultimo(), Primero(), Ultimo(), Primero()]
    counter = 0
    
    for future in as_completed(tareas):
        n = "la tarea mas rapida" if counter == 0 else "la siguiente tarea mas rapida"
        counter += 1
        result = await future
        print(f"{n} obtuvo el resultado: {result}")

await iniciar_as_completed()

# <center> **CREATE_TASK**</center>

In [ ]:
# El siguiente ejemplo demuestra cómo convertir una rutina en una tarea y programarla en el bucle de eventos.

from asyncio import create_task

async def iniciar_create_task():
    
    tarea = create_task(Primero())
    print(tarea)
    
    await asyncio.sleep(2)
    print("MAS PROCESOS!")
    
    await asyncio.sleep(3)
    print(sum(tarea))
    
    return tarea

await iniciar_create_task()

# <center> **POOLS**</center>
# <center> **GET_RUNNING_LOOP**</center>

## <center> **ASYNCIO Event Loop - MULTI HILO**</center>

In [ ]:
# importa la libtreria
from asyncio import get_running_loop
# importa librerias para trabajar concurrencia
from concurrent.futures import ThreadPoolExecutor

def blocking_io(value = 23, otros = 10):
    # File operations (such as logging) can block the
    # event loop: run them in a thread pool.
    with open("/dev/urandom", "rb") as f:
        return f.read(value + otros)
    
async def asyncio_multi_hilo():
    
    loop = get_running_loop()
    pool = ThreadPoolExecutor()
    
    # tipo de pool, funcion, parametros
    return await loop.run_in_executor(pool, blocking_io, 50, 70)

await asyncio_multi_hilo()

## <center> **ASYNCIO - MULTI PROCESO**</center>

In [ ]:
from concurrent.futures import ProcessPoolExecutor

def cpu_bound(value = 23, otros = 10):
    # CPU-bound operations will block the event loop:
    # in general it is preferable to run them in a
    # process pool.
    return  sum(i * i for i in range(10 ** 7)) // (value + otros)

    
async def asyncio_multi_proceso():
    
    loop = get_running_loop()
    pool = ProcessPoolExecutor()
    
    # tipo de pool, funcion, parametros
    return await loop.run_in_executor(pool, cpu_bound, 123, 100)

await asyncio_multi_proceso()

 # TABLA DE DEFINICIONES
 https://pharos.sh/concurrencia-en-python/#:~:text=al%20mismo%20tiempo.-,Simultaneidad%20vs%20paralelismo,acelerar%20el%20proceso%20de%20c%C3%A1lculo.
 

| CONCURRENCIA O SIMULTANEIDAD  | PARALELISMO |
| --- | --- |
| Es la ejecución de tareas al mismo tiempo, Cuando dos o más eventos son concurrentes, significa que están sucediendo al mismo tiempo. | Se logra cuando se realizan múltiples cálculos u operaciones al mismo tiempo o en paralelo con el objetivo de acelerar el proceso de cálculo. | 
| Solo tiene lugar en un procesador. | Utiliza múltiples procesadors para realizar tareas en paralelo. | 
| es la tarea de ejecutar y administrar múltiples cálculos al mismo tiempo. | es la tarea de ejecutar múltiples cálculos simultáneamente. |
| se logra a través de la operación de entrelazado de procesos en la unidad central de procesamiento (CPU) o, en otras palabras, mediante el cambio de contexto. | se logra a través de múltiples unidades centrales de procesamiento (CPU) |
| se puede realizar utilizando una sola unidad de procesamiento. | necesita múltiples unidades de procesamiento. |
| aumenta la cantidad de trabajo terminado a la vez. |  mejora el rendimiento y la velocidad computacional del sistema.|
| trata muchas cosas simultáneamente. | hace muchas cosas simultáneamente. |
|  es el enfoque de flujo de control no determinista.| es un enfoque de flujo de control determinista. |
|  la depuración es muy difícil. | la depuración también es difícil pero mas simple que la concurrencia. |



| HILO O SUBPROCESO | PROCESO | TAREA |
| --- | --- | --- |
| cada tarea ejecutada en un proceso es un subproceso. | Es un trabajo o una instancia de un programa calculado que se puede ejecutar. | Es un conjunto de instrucciones de programa que se cargan en la memoria.|
| Es la unidad de ejecución más pequeña que se puede realizar en una computadora. | Un hilo solo puede pertenecer a un proceso, pero un proceso puede tener múltiples hilos. |
| pueden acceder a los datos de otros subprocesos  | funcionan de forma aislada | |
| comparten memoria con otros subprocesos | cada proceso tiene su propia asignación de memoria. | |
||es un proceso liviano: en comparación con un proceso|||
| un subproceso genera menos carga en el sistema operativo para crear, mantener y administrar, lo que significa que el costo o la sobrecarga del subproceso es relativamente pequeño.|||
| El hilo no tiene espacio de direcciones, y el hilo está contenido en el espacio de direcciones del proceso.|||
| Todos los hilos comparten la memoria y los recursos del proceso. |||


| HILOS | PROCESOS | ASINCRONISMO |
| --- | --- | --- |
| Implementa concurrencia a través de hilos de aplicación | Implementa la concurrencia usando procesos del sistema | Construye aplicaciones concurrentes que utilizan co-rutinas. Utiliza un enfoque de un solo hilo y un solo proceso en el que partes de una aplicación cooperan para cambiar tareas explícitamente en momentos óptimos.|
|Multihilo es el proceso en el que multiples hilos se ejecutan al mismo tiempo en un proceso.|El multiproceso es permitir que se realicen múltiples procesos al mismo tiempo,  utilizando  uno o mas procesadores.|